Download [lex.mturk](https://cs.pomona.edu/~dkauchak/simplification/lex.mturk.14/lex.mturk.14.tar.gz)
 and extract sentences, targets, and candidates 

In [ ]:
import pickle
with open('task_data/lex.mturk.txt','r',encoding='iso-8859-1') as f:
     # open('task_data/lex_mturk_sen.txt','w') as f_sen,\
     # open('task_data/targets.pickle','wb') as f_targets,\
     # open('task_data/candidates.pickle','wb') as f_candidates:
     # open('task_data/pos_tags.pickle','wb') as f_pos_tags:
    f_str = f.readline()
    sens,targets,candidates,pos_tags = [],[],[],[]
    for l in f:
        sen,target,*candidate = l.strip().split('\t')
        # sen_pos = nltk.pos_tag(sen.split())
        # id = [w for w,tag in sen_pos].index(target)
        # target_pos = [tag for w,tag in sen_pos][id]
        # f_sen.write(sen+'\n')
        # pos_tags.append(target_pos)
        sens.append(sen.strip('"').split())
        targets.append(target)
        candidates.append(set(candidate))
    # pickle.dump(targets,f_targets,protocol=pickle.HIGHEST_PROTOCOL)
    # pickle.dump(candidates,f_candidates,protocol=pickle.HIGHEST_PROTOCOL)
    # pickle.dump(pos_tags,f_pos_tags,pickle.HIGHEST_PROTOCOL)

check whether targets in the sentences

In [ ]:
for i,(target,sen) in enumerate(zip(targets,sens)):
    if target not in sen:
        print(sen)
        print(targets)
        print(i)
        break

correct the sentences

In [ ]:
sens[111][0] = sens[111][0].lower()
sens[187][0] = sens[187][0].lower()
sens[209][0] = sens[209][0].lower()
sens[301][0] = sens[301][0].lower()

obtain the pos tags of targets

In [ ]:
from pathlib import Path
import stanza

In [5]:
# prepare pos tags of target words
nlp = stanza.Pipeline('en',processors='tokenize,pos',tokenize_pretokenized=True)
doc = nlp(sens)

2020-06-02 16:28:56 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-06-02 16:28:56 INFO: Use device: gpu
2020-06-02 16:28:56 INFO: Loading: tokenize
2020-06-02 16:28:56 INFO: Loading: pos
2020-06-02 16:28:59 INFO: Done loading processors!


In [6]:
pos_tags = []
for doc_sentence,target,sen in zip(doc.sentences,targets,sens):
    id = sen.index(target)
    word = doc_sentence.words[id]
    pos_tags.append(word.xpos)

check whether candidates have phrases

In [ ]:
for i,candidate in enumerate(candidates):
    for c in candidate:
        if len(c.split()) >= 2:
            print(candidate)
            print(i)
            break

remove phrases since

1. word2vec only contains tokens
2. words in constrain are not phrases

In [ ]:
w_candidates = []
for candidate in candidates:
    # only preserve tokens that are not phrases
    w_candidates.append({c for c in candidate if len(c.split()) == 1})

chech whether candidate words have spaces in the end of the word


In [ ]:
for candidate in w_candidates:
    for c in candidate:
        if c[-1] == ' ':
            print(c)
            print(candidate)
            break

remove the spaces
 

In [ ]:
w_candidates_ = []
for candidate in w_candidates:
    w_candidates_.append({c.strip(' ') for c in candidate})

lemmatize the target and the candidates

In [39]:
from nltk import WordNetLemmatizer
from utils import penn2morphy, get_all_words_covered_in_constraint, generate_sub_thesauri
lemmatizer = WordNetLemmatizer()

In [10]:
lemma_targets,lemma_candidates = [],[]
for target,cand_set,tag in zip(targets,w_candidates_,pos_tags):
    tag_ = penn2morphy(tag)
    lemma_cand_set = set()
    lemma_targets.append(lemmatizer.lemmatize(target,tag_)) 
    for c in cand_set:
        lemma_cand_set.add(lemmatizer.lemmatize(c,tag_))
    lemma_candidates.append(lemma_cand_set)


check the candidates coverage

In [13]:
set_candidates = set()
for candidate in lemma_candidates:
    set_candidates |= candidate

In [14]:
len(set_candidates)

2616

In [12]:
constrain_vocab = get_all_words_covered_in_constraint(['synonyms.txt','antonyms.txt'],set_candidates)

In [13]:
len(set_candidates & constrain_vocab),len(set_candidates)

2310

check the targets & candidates coverage

In [15]:
set_targets_candidates =  set(lemma_targets) | set_candidates

In [15]:
constrain_vocab = get_all_words_covered_in_constraint(['synonyms.txt','antonyms.txt'],set_targets_candidates)

In [16]:
len(set_targets_candidates & constrain_vocab),len(set_targets_candidates)

(2549, 2848)

In [37]:
len(set(lemma_targets) & constrain_vocab),len(set(lemma_targets)) 

(432, 441)

In [38]:
len(set_candidates & constrain_vocab),len(set(set_candidates)) 

(2325, 2616)

investigate that words are not covered

In [ ]:
import random

In [18]:
rest_vocab = set_targets_candidates - (set_targets_candidates & constrain_vocab)

In [36]:
random.sample(rest_vocab,2)

['sepcial', 'noticable']

replace the target in sentences with lemma_targets 

In [ ]:
for sen,target,lemma_target in zip(sens,targets,lemma_targets):
    id = sen.index(target)
    sen[id] = lemma_target

now prepare the final vocab, constrain_vocab, and embeddings

In [ ]:
vocab = set()
for sen in sens:
    vocab.update(sen)
vocab |= set_targets_candidates

In [17]:
len(vocab)

6278

In [41]:
from preprocess import GeneralTextProcesser
from constants import ORIGINAL_VECS_DIR, ORIGINAL_EMBEDDING, THESAURUS_DIR

In [20]:
l_vocab = list(vocab)
text_preprocesser = GeneralTextProcesser()
emb_dict,_ = text_preprocesser.vocab2vec(l_vocab, ORIGINAL_VECS_DIR, 'lexical_simplification', ORIGINAL_EMBEDDING,
                                       ['pickle'], 'word2vec', normalize=False, oov_handle='none')

preparing vocab vectors
6278 words in vocab, 514 words not found in word embedding file, ignore them in the embedding
saving vocab vector file


In [21]:
final_vocab = set(emb_dict.keys())

In [22]:
len(final_vocab)

5764

In [23]:
constrain_vocab = get_all_words_covered_in_constraint(['synonyms.txt','antonyms.txt'],final_vocab)

In [34]:
# save constrain emb dict
constrain_emb_dict = {w:emb_dict[w] for w in constrain_vocab}
with open('lexical_simplification_constrain.pickle','wb') as f:
    pickle.dump(constrain_emb_dict,f,pickle.HIGHEST_PROTOCOL)

In [40]:
import numpy as np
from os.path import join

In [37]:
# save vocab
np.save('lexical_simplification.npy',list(final_vocab))
np.save('lexical_simplification_constrain.npy',list(constrain_vocab))

generate thesauri 

In [ ]:
generate_sub_thesauri(join(THESAURUS_DIR, 'synonyms.txt'),'synonyms.txt',final_vocab)
generate_sub_thesauri(join(THESAURUS_DIR, 'antonyms.txt'),'antonyms.txt',final_vocab)

save sentences, targets, candidates, and pos tags 

In [ ]:
# type(sens),type(lemma_targets),type(lemma_candidates),type(pos_tags)
with open('lex_mturk_sen.pickle','wb') as f:
    pickle.dump(sens,f,pickle.HIGHEST_PROTOCOL)
with open('targets.pickle','wb') as f:
    pickle.dump(lemma_targets,f,pickle.HIGHEST_PROTOCOL)
with open('candidates.pickle','wb') as f:
    pickle.dump(lemma_candidates,f,pickle.HIGHEST_PROTOCOL)
with open('pos_tags.pickle','wb') as f:
    pickle.dump(pos_tags,f,pickle.HIGHEST_PROTOCOL)